In [1]:
from pytorch_forecasting.data.examples import get_stallion_data
import pyarrow
import fastparquet
import numpy as np
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, IterableDataset
import torch.nn as nn

In [2]:
data = get_stallion_data()

In [3]:
data.head()

,agency,sku,volume,date,industry_volume,soda_volume,avg_max_temp,price_regular,price_actual,discount,...,labor_day,independence_day,revolution_day_memorial,regional_games,fifa_u_17_world_cup,football_gold_cup,beer_capital,music_fest,discount_in_percent,timeseries
0,Agency_22,SKU_01,52.2720,2013-01-01,492612703,718394219,25.845238,1168.903668,1069.166193,99.737475,...,0,0,0,0,0,0,0,0,8.532566,0
238,Agency_37,SKU_04,0.0000,2013-01-01,492612703,718394219,26.505000,1852.273642,1611.466298,240.807344,...,0,0,0,0,0,0,0,0,13.000635,5
237,Agency_59,SKU_03,812.9214,2013-01-01,492612703,718394219,22.219737,1270.795012,1197.184260,73.610752,...,0,0,0,0,0,0,0,0,5.792496,9
236,Agency_11,SKU_01,316.4400,2013-01-01,492612703,718394219,25.360000,1176.155397,1082.757488,93.397909,...,0,0,0,0,0,0,0,0,7.940950,14
235,Agency_05,SKU_05,420.9093,2013-01-01,492612703,718394219,24.079012,1327.003396,1207.822992,119.180404,...,0,0,0,0,0,0,0,0,8.981168,22


In [4]:
# add time index
data["time_idx"] = data["date"].dt.year * 12 + data["date"].dt.month

data["time_idx"] -= data["time_idx"].min()
# add additional features

# show sample data
data.sample(10, random_state=521)

,agency,sku,volume,date,industry_volume,soda_volume,avg_max_temp,price_regular,price_actual,discount,...,independence_day,revolution_day_memorial,regional_games,fifa_u_17_world_cup,football_gold_cup,beer_capital,music_fest,discount_in_percent,timeseries,time_idx
291,Agency_25,SKU_03,0.5076,2013-01-01,492612703,718394219,25.845238,1264.162234,1152.473405,111.688829,...,0,0,0,0,0,0,0,8.835008,228,0
871,Agency_29,SKU_02,8.7480,2015-01-01,498567142,762225057,27.584615,1316.098485,1296.804924,19.293561,...,0,0,0,0,0,0,0,1.465966,177,24
19532,Agency_47,SKU_01,4.9680,2013-09-01,454252482,789624076,30.665957,1269.250000,1266.490490,2.759510,...,1,0,0,0,0,0,0,0.217413,322,8
2089,Agency_53,SKU_07,21.6825,2013-10-01,480693900,791658684,29.197727,1193.842373,1128.124395,65.717978,...,0,0,0,0,0,1,0,5.504745,240,9
9755,Agency_17,SKU_02,960.5520,2015-03-01,515468092,871204688,23.608120,1338.334248,1232.128069,106.206179,...,0,0,0,0,0,0,1,7.935699,259,26
7561,Agency_05,SKU_03,1184.6535,2014-02-01,425528909,734443953,28.668254,1369.556376,1161.135214,208.421162,...,0,0,0,0,0,0,0,15.218151,21,13
19204,Agency_11,SKU_05,5.5593,2017-08-01,623319783,1049868815,31.915385,1922.486644,1651.307674,271.178970,...,0,0,0,0,0,0,0,14.105636,17,55
8781,Agency_48,SKU_04,4275.1605,2013-03-01,509281531,892192092,26.767857,1761.258209,1546.059670,215.198539,...,0,0,0,0,0,0,1,12.218455,151,2
2540,Agency_07,SKU_21,0.0000,2015-10-01,544203593,761469815,28.987755,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0.000000,300,33
12084,Agency_21,SKU_03,46.3608,2017-04-01,589969396,940912941,32.478910,1675.922116,1413.571789,262.350327,...,0,0,0,0,0,0,0,15.654088,181,51


In [5]:
data['time_idx'].max()

59

In [6]:
data.loc[(data['agency'] == 'Agency_22') & (data['sku'] == 'SKU_01')].\
    sort_values(by = 'time_idx')[['agency', 'sku', 'date', 'volume']].head()

,agency,sku,date,volume
0,Agency_22,SKU_01,2013-01-01,52.272
7096,Agency_22,SKU_01,2013-02-01,62.532
8898,Agency_22,SKU_01,2013-03-01,74.196
10733,Agency_22,SKU_01,2013-04-01,89.424
12472,Agency_22,SKU_01,2013-05-01,79.164


In [7]:
data.columns

Index(['agency', 'sku', 'volume', 'date', 'industry_volume', 'soda_volume',
       'avg_max_temp', 'price_regular', 'price_actual', 'discount',
       'avg_population_2017', 'avg_yearly_household_income_2017', 'easter_day',
       'good_friday', 'new_year', 'christmas', 'labor_day', 'independence_day',
       'revolution_day_memorial', 'regional_games', 'fifa_u_17_world_cup',
       'football_gold_cup', 'beer_capital', 'music_fest',
       'discount_in_percent', 'timeseries', 'time_idx'],
      dtype='object')

In [8]:
data['agency'].nunique()

58

In [9]:
data['sku'].nunique()

25

In [10]:
data.loc[data['time_idx'] <= 53].shape

(18900, 27)

In [11]:
class InstockMask(nn.Module):
    def __init__(self, time_step, ltsp, min_instock_ratio = 0.5, eps_instock_dph = 1e-3,
                eps_total_dph = 1e-3, **kwargs):

        super(InstockMask, self).__init__(**kwargs)

        if not eps_total_dph > 0:
            raise ValueError(f"epsilon_total_dph of {eps_total_dph} is invalid! \
                              This parameter must be > 0 to avoid division by 0.")

        self.min_instock_ratio = min_instock_ratio
        self.eps_instock_dph = eps_instock_dph
        self.eps_total_dph = eps_total_dph

    def forward(self, demand, total_dph, instock_dph):

        if total_dph is not None and instock_dph is not None:

            total_dph = total_dph + self.eps_total_dph
            instock_dph = instock_dph + self.eps_instock_dph
            instock_rate = torch.round(instock_dph/total_dph)

            demand = torch.where(instock_rate >= self.min_instock_ratio, demand,
                                 -torch.ones_like(demand))

        return demand


class _BaseInstockMask(nn.Module):      
    def __init__(self, time_step, ltsp, min_instock_ratio = 0.5, eps_total_dph = 1e-3,
                eps_instock_dph = 1e-3, **kwargs):

        super(_BaseInstockMask, self).__init__(**kwargs)

        if not eps_total_dph > 0:
            raise ValueError(f"epsilon_total_dph of {eps_total_dph} is invalid! \
                              This parameter must be > 0 to avoid division by 0.")

        self.instock_mask = InstockMask(time_step, ltsp, min_instock_ratio=min_instock_ratio,
                                        eps_instock_dph = eps_instock_dph, 
                                        eps_total_dph = eps_total_dph)

    def forward(self):
        raise NotImplementedError

class HorizonMask(_BaseInstockMask):
    def __init__(self, time_step, ltsp, min_instock_ratio = 0.5, eps_instock_dph=1e-3,
                eps_total_dph=1e-3, **kwargs):

        super(HorizonMask, self).__init__(time_step, ltsp, 
                                          min_instock_ratio = min_instock_ratio,
                                          eps_instock_dph=eps_instock_dph,
                                          eps_total_dph=eps_total_dph, **kwargs)
        
        self.mask_idx = _compute_horizon_mask(time_step, ltsp)

    def forward(self, demand, total_dph, instock_dph):
        demand_instock = self.instock_mask(demand, total_dph, instock_dph).float()
        
        mask = mask_idx.repeat(demand_instock.shape[0], 1, 1)
        
        print(f'demand shape: {demand_instock.shape}, mask shape: {mask_idx.shape}')
        masked_demand = torch.where(mask, demand_instock, -torch.ones_like(demand_instock))

        return masked_demand
        

def _compute_horizon_mask(time_step, ltsp):

    horizon = np.array(list(map(lambda _ltsp: _ltsp[0] + _ltsp[1], ltsp))).\
                        reshape((1, len(ltsp)))

    forecast_date_range = np.arange(time_step).reshape((time_step, 1))
    relative_distance = forecast_date_range + horizon
    mask = relative_distance < time_step
    return torch.tensor(mask).unsqueeze(0)

In [12]:
class DemandExpander(nn.Module):

    def __init__(self, time_step, ltsp, normalize = True,
                mask_func = HorizonMask, min_instock_ratio=0.5,
                eps_instock_dph = 1e-3, eps_total_dph = 1e-3, **kwargs):

        super(DemandExpander, self).__init__(**kwargs)
        if not eps_total_dph > 0:
            raise ValueError("eps_total_dph can't be 0")

        Tpred = max(map(lambda x: x[0] + x[1], ltsp))
        pos_sp1 = [i for i, x in enumerate(ltsp) if x[1] == 1]
        pos_spN = [i for i, x in enumerate(ltsp) if x[1] != 1]

        self.pos_sp1 = pos_sp1
        self.pos_spN = pos_spN

        self.ltsp_kernel = _ltsp_kernel(Tpred, ltsp, normalize)
        self.ltsp_idx = _ltsp_idx(time_step, Tpred)
        self.demand_mask = mask_func(time_step, ltsp, min_instock_ratio=min_instock_ratio,
                                     eps_instock_dph=eps_instock_dph,
                                     eps_total_dph = eps_total_dph)

    def forward(self, demand, total_dph, instock_dph):
        ltsp_demand = _apply_ltsp_kernel(demand, self.ltsp_idx, self.ltsp_kernel)

        ltsp_idph = _apply_ltsp_kernel(instock_dph, self.ltsp_idx, self.ltsp_kernel)
        ltsp_dph = _apply_ltsp_kernel(total_dph, self.ltsp_idx, self.ltsp_kernel)

        masked_demand = self.demand_mask(ltsp_demand, ltsp_dph, ltsp_idph)
        masked_demand_sp1 = masked_demand[:, :, self.pos_sp1]
        masked_demand_spN = masked_demand[:, :, self.pos_spN]

        return masked_demand_sp1, masked_demand_spN

def _ltsp_idx(time_step, Tpred):
        idx = np.arange(time_step).reshape(-1, 1) + np.arange(Tpred)
        return torch.tensor(idx)

def _ltsp_kernel(Tpred, ltsp, normalize = True):
    
        ltsp_count = len(ltsp)
        kernel = np.zeros((Tpred, ltsp_count), dtype = 'float32')
        for i in range(len(ltsp)):
            lead_time = ltsp[i][0]
            span = ltsp[i][1]
            if normalize:
                kernel[lead_time:lead_time + span, i] = 1.0/span
            else:
                kernel[lead_time:lead_time + span, i] = 1.0

        return torch.tensor(kernel)

def _apply_ltsp_kernel(s, ltsp_idx, ltsp_kernel):
        s_ltsp = s[:, ltsp_idx].float()
        
        return s_ltsp @ ltsp_kernel 

In [13]:
class Dataset(Dataset):
    
    def __init__(self, data, static_features, timevarying_features, future_information, 
                 target, train_time_step, predict_time_step, num_quantiles, ltsp):
        
        self.data = data
        self.train_time_step = train_time_step
        self.predict_time_step = predict_time_step
        self.num_quantiles = num_quantiles
        
        self.ltsp_kernel = _ltsp_kernel(predict_time_step, ltsp)
        
        self.ltsp_idx = _ltsp_idx(time_step = train_time_step, Tpred = predict_time_step)
        
        
        self.static_features = torch.tensor(self.data.\
                              loc[self.data['time_idx'] < self.train_time_step][static_features].\
                to_numpy(np.float64).reshape(-1, self.train_time_step, len(static_features))).float()
        
        self.timevarying_features = torch.tensor(self.data.\
                              loc[self.data['time_idx'] < self.train_time_step][timevarying_features].\
                to_numpy(np.float64).reshape(-1, self.train_time_step, len(timevarying_features))).float()
            
        self.future_information = torch.tensor(self.data[future_information].\
                to_numpy(np.float64).reshape(-1, (self.train_time_step + self.predict_time_step), len(future_information))).float()
        
        self.targets = torch.tensor(self.data[target].\
            to_numpy(np.float64).reshape(-1, (self.train_time_step + self.predict_time_step))).float()
        
        self.targets = _apply_ltsp_kernel(self.targets, self.ltsp_idx, self.ltsp_kernel)
        
    def __len__(self):
        
        return self.timevarying_features.shape[1]
    
    def __getitem__(self, idx):
        
        static_features = self.static_features[idx, :, :]
        timevarying_features = self.timevarying_features[idx, :, :]
        future_information = self.future_information[idx, :, :]
        targets = self.targets[idx, :, :]
        
        return dict(static_features = static_features, timevarying_features = timevarying_features,
                    future_information = future_information, targets = targets)
    
def _ltsp_idx(time_step, Tpred):
        idx = np.arange(time_step).reshape(-1, 1) + np.arange(Tpred)
        return torch.tensor(idx)

def _ltsp_kernel(Tpred, ltsp, normalize = True):
    
        ltsp_count = len(ltsp)
        kernel = np.zeros((Tpred, ltsp_count), dtype = 'float32')
        for i in range(len(ltsp)):
            lead_time = ltsp[i][0]
            span = ltsp[i][1]
            if normalize:
                kernel[lead_time:lead_time + span, i] = 1.0/span
            else:
                kernel[lead_time:lead_time + span, i] = 1.0

        return torch.tensor(kernel)

def _apply_ltsp_kernel(s, ltsp_idx, ltsp_kernel):
        s_ltsp = s[:, ltsp_idx].float()
        
        return s_ltsp @ ltsp_kernel 

In [14]:
ltsp = [(i, 1) for i in range(6)]
len(ltsp)

6

In [15]:
data['month'] = data['date'].dt.month

In [16]:
data_sorted = data.sort_values(['agency', 'sku', 'date'])

In [17]:
data_sorted = pd.get_dummies(data_sorted, columns=['month'])

In [18]:
training = Dataset(data_sorted,
                   static_features=['avg_population_2017'],
                   timevarying_features=['volume', 'industry_volume', 'soda_volume', 'price_regular'],
                   future_information=['month_1', 'month_2',
                                       'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
                                       'month_9', 'month_10', 'month_11', 'month_12', 'price_regular'],
                   target=['volume'], 
                   train_time_step=54, 
                   predict_time_step=6,
                   num_quantiles = 2,
                   ltsp = ltsp)

In [19]:
loader = DataLoader(training, 4)

In [20]:
for batch in loader:
    print(f"static_features {batch['static_features'].shape}")
    print(f"timevarying_features {batch['timevarying_features'].shape}")
    print(f"future_information {batch['future_information'].shape}")
    print(f"targets {batch['targets'].shape}")
    print("---------")

static_features torch.Size([4, 54, 1])
timevarying_features torch.Size([4, 54, 4])
future_information torch.Size([4, 60, 13])
targets torch.Size([4, 54, 6])
---------
static_features torch.Size([4, 54, 1])
timevarying_features torch.Size([4, 54, 4])
future_information torch.Size([4, 60, 13])
targets torch.Size([4, 54, 6])
---------
static_features torch.Size([4, 54, 1])
timevarying_features torch.Size([4, 54, 4])
future_information torch.Size([4, 60, 13])
targets torch.Size([4, 54, 6])
---------
static_features torch.Size([4, 54, 1])
timevarying_features torch.Size([4, 54, 4])
future_information torch.Size([4, 60, 13])
targets torch.Size([4, 54, 6])
---------
static_features torch.Size([4, 54, 1])
timevarying_features torch.Size([4, 54, 4])
future_information torch.Size([4, 60, 13])
targets torch.Size([4, 54, 6])
---------
static_features torch.Size([4, 54, 1])
timevarying_features torch.Size([4, 54, 4])
future_information torch.Size([4, 60, 13])
targets torch.Size([4, 54, 6])
--------

In [21]:
demand = torch.tensor(np.random.choice(100, (4, 60)))

In [22]:
total_dph = torch.tensor(24 * np.ones((4, 60)).reshape(4, 60))

In [23]:
idph = torch.tensor(np.random.choice(24, (4, 60)).reshape(4, 60))

In [24]:
d = DemandExpander(54, ltsp)

In [25]:
mask_idx = _compute_horizon_mask(54, ltsp)

In [26]:
mask_idx.shape

torch.Size([1, 54, 6])

In [27]:
masked_demand = d(demand, total_dph, idph)

demand shape: torch.Size([4, 54, 6]), mask shape: torch.Size([1, 54, 6])


In [28]:
masked_demand[0].shape

torch.Size([4, 54, 6])

In [29]:
targets = torch.rand(350, 60).float()

In [30]:
idx = torch.rand(54, 6).long()
ltsp_kernel = torch.rand(6,6)

In [31]:
t = targets[:, idx] @ ltsp_kernel

In [32]:
t.shape

torch.Size([350, 54, 6])